In [131]:
import os
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams['figure.figsize']=(10,10)
import math
import time
import pandas as pd
import numpy as np
import scipy
import sklearn
import seaborn as sns
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )
from copy import deepcopy

In [132]:
dat=pd.read_csv('/Users/tony/Desktop/Jet_infosystems/Примеры для приема.csv',encoding='utf-8',error_bad_lines=False,sep=';',decimal=',')
dat.rename(columns=dict(zip(dat.columns,['market','receipt_number','receipt_date','number','price','summa','serial_number','worker','summa_balov','bonus_card','receipt_type'])),inplace=True)
dat=dat.ix[~dat.worker.isnull(),:]

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [133]:
dat.head()

,market,receipt_number,receipt_date,number,price,summa,serial_number,worker,summa_balov,bonus_card,receipt_type,strTypeDirr
0,0x00003DF3FDACC72FD85F505BCCE8377A,PCRC11198839,2015-05-03 11:04:16.800,1.0,2579.0,2579.0,0,0xB6FB1975EE439EA5CD19A5A96AD16F6D,0,NaN,Продажа,Платеж
1,0x00003DF3FDACC72FD85F505BCCE8377A,PCRC11198880,2015-05-03 19:25:16.800,1.0,63000.0,63000.0,0,0x779266FA890AE3707AAD3FAC8D740DD0,0,NaN,Продажа,Платеж
2,0x00003DF3FDACC72FD85F505BCCE8377A,PCRC11198973,2015-05-05 18:05:16.800,1.0,0.0,0.0,0,0xB0587EA47B000DE3B267B4318BD6C854,0,NaN,Продажа,Аксессуар
3,0x00003DF3FDACC72FD85F505BCCE8377A,PCRC11198896,2015-05-04 11:00:16.800,1.0,5050.0,5050.0,0,0xB6FB1975EE439EA5CD19A5A96AD16F6D,0,NaN,Продажа,Платеж
4,0x00003DF3FDACC72FD85F505BCCE8377A,PCRC11198871,2015-05-03 17:12:16.800,1.0,1300.0,1300.0,0,0xB0587EA47B000DE3B267B4318BD6C854,0,NaN,Продажа,Платеж


## Исследование суммы и цены

Сумма $\not =$ Цена $\times$ Кол-во

In [134]:
dat.ix[(dat.price*dat.number!=dat.summa)&(dat.bonus_card.isnull()),:].head()

,market,receipt_number,receipt_date,number,price,summa,serial_number,worker,summa_balov,bonus_card,receipt_type,strTypeDirr
30,0x00003DF3FDACC72FD85F505BCCE8377A,PCRC11198973,2015-05-05 18:05:16.800,1.0,990.0,792.0,0,0xB0587EA47B000DE3B267B4318BD6C854,0,NaN,Продажа,Аксессуар
57,0x00003DF3FDACC72FD85F505BCCE8377A,PCRC11199228,2015-05-11 18:10:16.800,1.0,1490.0,1192.0,0,0xB6FB1975EE439EA5CD19A5A96AD16F6D,0,NaN,Продажа,Аксессуар
202,0x0001C9CA7CDB25F0A9D987C8FC5D6EF8,PCRC80807946,2015-04-18 11:48:03.800,1.0,3190.0,2990.0,6623030,0x08AAF9811BF20CD7709854F825B2F027,0,NaN,Продажа,Телефон
215,0x0001C9CA7CDB25F0A9D987C8FC5D6EF8,PCRC80808892,2015-05-15 13:20:16.800,1.0,2290.0,1832.0,0,0xA31DD0E25A91EEDEBF6DB958FB2214EF,0,NaN,Продажа,Телефон
227,0x0001C9CA7CDB25F0A9D987C8FC5D6EF8,PCRC80808901,2015-05-15 15:04:16.800,1.0,2290.0,1832.0,0,0xA31DD0E25A91EEDEBF6DB958FB2214EF,0,NaN,Продажа,Телефон


Возможно у покупателя была бонусная карта, или в этом магазине на этот товар в тот период действовала акция, но в разделе "бонусная карта" таких записей нет. При условии того, что во всём наборе данных бонусная карта содержится только в ~1% случаев весьма вероятно, что это не мошенничество.

В таком случае, если это всё же скидка, то скидки обычно делают целыми: 10%, 15% и т.д. Проверим, так ли это.

In [135]:
dat.insert(loc=6,column='skidka_perc',value=(1-dat.summa/(dat.price*dat.number)).values.round(decimals=2))
dat.insert(loc=7,column='skidka_rub',value=(-dat.summa+dat.price).values.round(decimals=2))

In [136]:
dat.skidka_perc[(dat.price*dat.number!=dat.summa)&(dat.bonus_card.isnull())].value_counts(1).head(10)

0.20    0.765170
0.25    0.033070
0.15    0.023362
0.13    0.021541
0.18    0.013350
1.00    0.012439
0.10    0.011833
0.08    0.010922
0.04    0.010012
0.07    0.008192
Name: skidka_perc, dtype: float64

В 77% случаях это действительно скидка в размере 20%. Но были и странные "скидки".

In [137]:
dat.skidka_perc[(dat.price*dat.number!=dat.summa)&(dat.bonus_card.isnull())].value_counts(1).tail(10)

 0.16    0.000303
 0.88    0.000303
 0.79    0.000303
 0.93    0.000303
 0.82    0.000303
-0.12    0.000303
 0.98    0.000303
 0.97    0.000303
-0.09    0.000303
-9.03    0.000303
Name: skidka_perc, dtype: float64

Отрицательная скидка, индикатор мошенничества!

In [138]:
dat.ix[(dat.skidka_perc<0),:]

,market,receipt_number,receipt_date,number,price,summa,skidka_perc,skidka_rub,serial_number,worker,summa_balov,bonus_card,receipt_type,strTypeDirr
56383,0x14466CB7D3B0D9B891CD587D3C30C84B,PCRIT090000160,2009-04-15 15:58:08.000,1.0,2990.0,29990.0,-9.03,-27000.0,1111005,0x5C4D28B735A317E90338CEAF7E0CC0AC,0,NaN,Возврат,телефон
67105,0x16A4701668D5B25A6A8826A2C2926BDD,PCRC1020834,2015-04-08 15:32:53.800,1.0,800.0,900.0,-0.12,-100.0,3155932,0x7BFC984DC6528D723F7489ADEA6C787F,0,NaN,Продажа,Телефон
676561,0x14466CB7D3B0D9B891CD587D3C30C84B,PCRD0160028733,2009-04-24 05:43:16.000,1.0,1.0,199.0,-198.00,-198.0,5.38984e+06,0x5C4D28B735A317E90338CEAF7E0CC0AC,0,NaN,Возврат,телефон
699119,0x53861306B790E59740A0CAE4B1906D8B,PCRP6290045222,2009-04-20 17:27:25.000,1.0,10990.0,11990.0,-0.09,-1000.0,9.75617e+06,0xE07DC0BD4D81B6EF4C9649ADD533EF37,0,NaN,Возврат,телефон


Здесь продавец 0x5C4D28B735A317E90338CEAF7E0CC0AC дважды засветился как нечестный на руку. Заметим, что он оба раз принимал "возврат". А есть ли такие продавцы, которые принимали больше возвратов, чем продавали: $\dots$

In [139]:
dic={'Продажа':1,"Возврат":0}
dat['type_enc']=dat['receipt_type'].apply(lambda x: dic[x])

In [140]:
dat.ix[dat.receipt_type=='Возврат','worker'].value_counts().head(7)

0x5C4D28B735A317E90338CEAF7E0CC0AC    94
0x6A5A25E9677782A85559EB48FE88079C     7
0xACB280BD53F018D351B0C0A3F6A4A297     4
0xCA729AB217066F9E4FAAADE116F1DBD6     3
0x1066C214465E11A87C5107BFE3567321     3
0x8C6BAE582FB320CD23DF14C715774780     3
0xF4D2657DEF529AA2104D6BA5DE34AAA2     2
Name: worker, dtype: int64

In [141]:
dat.receipt_type[dat.worker=='0x5C4D28B735A317E90338CEAF7E0CC0AC'].value_counts()

Возврат    94
Продажа     1
Name: receipt_type, dtype: int64

У продавца 0x5C4D28B735A317E90338CEAF7E0CC0AC несоизмеримо больше возвратов, чем продаж и это подозрительно, потому что он такой один.

In [142]:
dat.ix[(dat.receipt_type=='Возврат')&(dat.worker=='0x5C4D28B735A317E90338CEAF7E0CC0AC')&(dat.skidka_rub!=0),:]

,market,receipt_number,receipt_date,number,price,summa,skidka_perc,skidka_rub,serial_number,worker,summa_balov,bonus_card,receipt_type,strTypeDirr,type_enc
56383,0x14466CB7D3B0D9B891CD587D3C30C84B,PCRIT090000160,2009-04-15 15:58:08.000,1.0,2990.0,29990.0,-9.03,-27000.0,1111005,0x5C4D28B735A317E90338CEAF7E0CC0AC,0,NaN,Возврат,телефон,0
59136,0x14466CB7D3B0D9B891CD587D3C30C84B,PCRC3100023000,2009-04-18 12:07:44.000,1.0,4990.0,15.0,1.00,4975.0,9551807,0x5C4D28B735A317E90338CEAF7E0CC0AC,0,NaN,Возврат,телефон,0
676561,0x14466CB7D3B0D9B891CD587D3C30C84B,PCRD0160028733,2009-04-24 05:43:16.000,1.0,1.0,199.0,-198.00,-198.0,5.38984e+06,0x5C4D28B735A317E90338CEAF7E0CC0AC,0,NaN,Возврат,телефон,0


Как-то у этого продавца подозрительно скачут "цена-сумма"

In [143]:
procent=dat.skidka_perc[(dat.price*dat.number!=dat.summa)&(dat.bonus_card.isnull())].value_counts(1).keys()

Рассмотрим все сделки со "странными" скидками. Странными их будем называть, если ни в процентах, ни в рублях число не кратно 5.

In [144]:
dat.ix[(((dat.skidka_rub%5)!=0)&(100*dat.skidka_perc.values.round(decimals=2)%5!=0))&dat.bonus_card.isnull()&dat.skidka_rub!=0,:].head(7)

,market,receipt_number,receipt_date,number,price,summa,skidka_perc,skidka_rub,serial_number,worker,summa_balov,bonus_card,receipt_type,strTypeDirr,type_enc
843,0x0045210B3169C431E5AEA4C9BE84A4E3,PCRC1104939,2015-04-17 16:33:45.800,1.0,9990.0,8695.91,0.13,1294.09,5277596,0xFD455FCC6A3592383590D79CADA1B53B,0,NaN,Продажа,Телефон,1
878,0x0045210B3169C431E5AEA4C9BE84A4E3,PCRC1104939,2015-04-17 16:33:45.800,1.0,650.0,565.86,0.13,84.14,0,0xFD455FCC6A3592383590D79CADA1B53B,0,NaN,Продажа,Телефон,1
891,0x0045210B3169C431E5AEA4C9BE84A4E3,PCRC1104939,2015-04-17 16:33:45.800,1.0,990.0,861.84,0.13,128.16,0,0xFD455FCC6A3592383590D79CADA1B53B,0,NaN,Продажа,Аксессуар,1
982,0x0045210B3169C431E5AEA4C9BE84A4E3,PCRC1104939,2015-04-17 16:33:45.800,1.0,1990.0,1732.39,0.13,257.61,0,0xFD455FCC6A3592383590D79CADA1B53B,0,NaN,Продажа,Телефон,1
8308,0x039A1B1599005B6ABD2FB72A2E33E4AE,PCRC11115159,2015-04-05 15:09:56.800,1.0,2390.0,1762.00,0.26,628.00,8080575,0x6639BC3C0FC1BB2E04E5A33E4A9508BD,0,NaN,Продажа,Телефон,1
33171,0x0CEEA1BF25C115937510D75ED7D8AC65,PCRC30509750,2015-04-16 19:06:13.800,1.0,7990.0,7666.74,0.04,323.26,812747,0x886716640A4F2597E24D0442365F11B8,0,NaN,Продажа,Телефон,1
33697,0x0CEEA1BF25C115937510D75ED7D8AC65,PCRC30509750,2015-04-16 19:06:13.800,1.0,690.0,662.08,0.04,27.92,0,0x886716640A4F2597E24D0442365F11B8,0,NaN,Продажа,Телефон,1


Среди них есть конечно и те, которые встречаются много раз (например скидки 12, 13 и 14%). Раз они много раз встречаются - вряд ли это выбросы. Посмотрим же на большие скидки (в денежном или процентном выражении).

In [145]:
x=dat.ix[(((dat.skidka_rub%5)!=0)&(100*dat.skidka_perc.values.round(decimals=2)%5!=0))&dat.bonus_card.isnull()&dat.skidka_rub!=0,:]

In [146]:
x.skidka_perc.value_counts().sort_index()

0.01     6
0.02     5
0.03     1
0.04     6
0.06     2
0.09     2
0.12    21
0.13    53
0.14    21
0.22     3
0.23     1
0.24     3
0.26     1
0.27     1
0.29     1
0.39     2
0.66     1
0.97     1
0.98     1
0.99     3
Name: skidka_perc, dtype: int64

In [147]:
x.ix[x.skidka_perc==0.98,:]

,market,receipt_number,receipt_date,number,price,summa,skidka_perc,skidka_rub,serial_number,worker,summa_balov,bonus_card,receipt_type,strTypeDirr,type_enc
58531,0x14466CB7D3B0D9B891CD587D3C30C84B,PCRY0870025901,2009-04-21 09:45:11.000,1.0,990.0,21.0,0.98,969.0,NaN,0xA4CFFB7C77D46915AF3D650B53A4A5C3,0,NaN,Продажа,Аксессуар,1


Сумма в 21 рубль это вообще странная вещь.

In [148]:
x.ix[x.skidka_perc==0.99,:]

,market,receipt_number,receipt_date,number,price,summa,skidka_perc,skidka_rub,serial_number,worker,summa_balov,bonus_card,receipt_type,strTypeDirr,type_enc
58457,0x14466CB7D3B0D9B891CD587D3C30C84B,PCRS1150021953,2009-04-17 11:00:04.000,1.0,9990.0,99.0,0.99,9891.0,0536808,0x82DCD089DC7C497DDFEF8808D9B6C40E,0,NaN,Продажа,телефон,1
216264,0x53861306B790E59740A0CAE4B1906D8B,PCRU1610000289,2009-04-15 19:03:54.000,1.0,490.0,4.0,0.99,486.0,5.82498e+06,0x73048B67439A5571AAB8FA52576DC11A,0,NaN,Возврат,телефон,0
631411,0xF8557E8750BA33EF48D875291A80F7CA,PCRC10204423,2015-05-04 12:42:16.800,1.0,190.0,1.0,0.99,189.0,0,0x7228E5AFE5E4E1CC5782F9D7439BC786,0,NaN,Продажа,Телефон,1


Все скидки больших размеров (99, 98, 97%) это скорее всего мошенничество. Первая строка показывает типичный пример мошенничества - дописать/убрать нолик (цена из 9990₽ превратилась в 99₽).

In [149]:
dat.ix[((dat.skidka_rub)/dat.summa>=10)&(dat.price!=0)&(dat.bonus_card.isnull()),:].head(7)

,market,receipt_number,receipt_date,number,price,summa,skidka_perc,skidka_rub,serial_number,worker,summa_balov,bonus_card,receipt_type,strTypeDirr,type_enc
563,0x0045210B3169C431E5AEA4C9BE84A4E3,PCRC1104258,2015-04-07 12:07:25.800,1.0,4490.0,0.0,1.0,4490.0,1658894,0x0F62F8ECD757FB623C64CB3C7EABF8EE,0,NaN,Продажа,Телефон,1
892,0x0045210B3169C431E5AEA4C9BE84A4E3,PCRC1104939,2015-04-17 16:33:45.800,1.0,1000.0,1.0,1.0,999.0,0,0xFD455FCC6A3592383590D79CADA1B53B,0,NaN,Продажа,Услуга по настройке,1
13635,0x05EE3F06EBCB01EB66479B1574A2186E,PCRC70202047,2015-04-16 13:03:55.800,1.0,1000.0,1.0,1.0,999.0,0,0xA74157CD9A68A2FDDEEE4EA1E3182976,0,NaN,Продажа,Услуга по настройке,1
14161,0x05EE3F06EBCB01EB66479B1574A2186E,PCRC70202048,2015-04-16 13:05:55.800,1.0,1000.0,1.0,1.0,999.0,0,0xA74157CD9A68A2FDDEEE4EA1E3182976,0,NaN,Продажа,Услуга по настройке,1
55831,0x14466CB7D3B0D9B891CD587D3C30C84B,PCRH1710011145,2009-04-16 15:36:32.000,1.0,4990.0,10.0,1.0,4980.0,8972699,0xB3235071F43DCEDB9C5E8875628CF5DF,0,NaN,Возврат,телефон,0
55833,0x14466CB7D3B0D9B891CD587D3C30C84B,PCRH1710011153,2009-04-16 15:48:04.000,1.0,150.0,0.0,1.0,150.0,NaN,0xB3235071F43DCEDB9C5E8875628CF5DF,0,NaN,Продажа,Аксессуар,1
55919,0x14466CB7D3B0D9B891CD587D3C30C84B,PCRY0870025901,2009-04-21 09:45:11.000,1.0,990.0,0.0,1.0,990.0,NaN,0xA4CFFB7C77D46915AF3D650B53A4A5C3,0,NaN,Продажа,Аксессуар,1


Если скидка в ряблях больше чем в 10 раз превышает сумму - признак мошенничества. Число 10 взято по сути с потолка, но видно что, проходящие по данному критерию строки, это примеры, где скидка радикально превышает сумму, при этом бонусной карты не зарегистрировано.

### Итак, некоторые признаки мошенничества:
1) Отрицательные скидки

2) "Странные" и большие скидки (88, 79, 93%)

3)  Соотношение кол-во продаж - кол-во возвратов у продавца

4) Величина скидки во много раз превышает итоговую сумму (при отсутствиии бонусной карты)

## Найденные примеры мошеннических строк в таблице

In [152]:
itog=list(dat.ix[((dat.skidka_rub)/dat.summa>=10)&(dat.price!=0)&(dat.bonus_card.isnull()),:].index)+list(x.ix[x.skidka_perc==0.99,:].index)+list([618])+list(dat.ix[(dat.receipt_type=='Возврат')&(dat.worker=='0x5C4D28B735A317E90338CEAF7E0CC0AC')&(dat.skidka_rub!=0),:].index)+list(dat.ix[(dat.skidka_perc<0),:].index)
dat.ix[np.sort(list(set(itog)-set([618])))]

,market,receipt_number,receipt_date,number,price,summa,skidka_perc,skidka_rub,serial_number,worker,summa_balov,bonus_card,receipt_type,strTypeDirr,type_enc
563,0x0045210B3169C431E5AEA4C9BE84A4E3,PCRC1104258,2015-04-07 12:07:25.800,1.0,4490.0,0.0,1.00,4490.0,1658894,0x0F62F8ECD757FB623C64CB3C7EABF8EE,0,NaN,Продажа,Телефон,1
892,0x0045210B3169C431E5AEA4C9BE84A4E3,PCRC1104939,2015-04-17 16:33:45.800,1.0,1000.0,1.0,1.00,999.0,0,0xFD455FCC6A3592383590D79CADA1B53B,0,NaN,Продажа,Услуга по настройке,1
13635,0x05EE3F06EBCB01EB66479B1574A2186E,PCRC70202047,2015-04-16 13:03:55.800,1.0,1000.0,1.0,1.00,999.0,0,0xA74157CD9A68A2FDDEEE4EA1E3182976,0,NaN,Продажа,Услуга по настройке,1
14161,0x05EE3F06EBCB01EB66479B1574A2186E,PCRC70202048,2015-04-16 13:05:55.800,1.0,1000.0,1.0,1.00,999.0,0,0xA74157CD9A68A2FDDEEE4EA1E3182976,0,NaN,Продажа,Услуга по настройке,1
55831,0x14466CB7D3B0D9B891CD587D3C30C84B,PCRH1710011145,2009-04-16 15:36:32.000,1.0,4990.0,10.0,1.00,4980.0,8972699,0xB3235071F43DCEDB9C5E8875628CF5DF,0,NaN,Возврат,телефон,0
55833,0x14466CB7D3B0D9B891CD587D3C30C84B,PCRH1710011153,2009-04-16 15:48:04.000,1.0,150.0,0.0,1.00,150.0,NaN,0xB3235071F43DCEDB9C5E8875628CF5DF,0,NaN,Продажа,Аксессуар,1
55919,0x14466CB7D3B0D9B891CD587D3C30C84B,PCRY0870025901,2009-04-21 09:45:11.000,1.0,990.0,0.0,1.00,990.0,NaN,0xA4CFFB7C77D46915AF3D650B53A4A5C3,0,NaN,Продажа,Аксессуар,1
56191,0x14466CB7D3B0D9B891CD587D3C30C84B,PCRD7570005691,2009-04-21 04:42:47.000,1.0,1790.0,53.0,0.97,1737.0,9085120,0xA8FAC05BFFAC4C39A5B80EDCCCAFE315,0,NaN,Продажа,телефон,1
56383,0x14466CB7D3B0D9B891CD587D3C30C84B,PCRIT090000160,2009-04-15 15:58:08.000,1.0,2990.0,29990.0,-9.03,-27000.0,1111005,0x5C4D28B735A317E90338CEAF7E0CC0AC,0,NaN,Возврат,телефон,0
56767,0x14466CB7D3B0D9B891CD587D3C30C84B,PCRC0580000139,2009-04-15 15:27:54.000,1.0,890.0,2.0,1.00,888.0,NaN,0x9A365FFD30093B7A56E508AD849B8F60,0,NaN,Продажа,Аксессуар,1
